In [2]:
import h5py
import numpy as np
import os 
import imageio
import glob

In [12]:
def h5_to_tiff(dirPath, filePath, output_tiff, chunk_size=100):
    """
    Convert a multi-frame HDF5 dataset to a multipage grayscale TIFF.
    
    Parameters:
        dirPath (str): Directory path where the HDF5 file is located.
        filePath (str): Name of the HDF5 file.
        output_tiff (str): Path for the output TIFF file.
        chunk_size (int): Number of frames to process at a time.
    """
    # Open the HDF5 file and read dataset shape
    with h5py.File(dirPath + filePath, 'r') as f_in:
        dset = f_in['data']
        total_frames, height, width = dset.shape
        print("Dataset shape:", dset.shape)
        
        # Open an imageio writer for TIFF in multi-image (append) mode.
        writer = imageio.get_writer(output_tiff, format='TIFF', mode='I')
        
        # Process the dataset in chunks to save memory
        for start in range(0, total_frames, chunk_size):
            end = min(start + chunk_size, total_frames)
            # Process each frame in this chunk
            for i in range(start, end):
                frame = dset[i, :, :]
                # If the frame is not uint8, scale it appropriately.
                if frame.dtype != np.uint8:
                    # Normalize frame data to 0-255 and convert to uint8.
                    frame = ((frame - frame.min()) / (frame.max() - frame.min()) * 255).astype(np.uint8)
                writer.append_data(frame)
                print(f"Processed frame {i+1} of {total_frames}")
        writer.close()
    print(f"Saved {total_frames} frames to {output_tiff} as a multipage TIFF.")

def h5_to_tiff_split(dirPath, filePath, output_dir, max_frames_per_tiff=1000, chunk_size=100):
    """
    Convert a multi-frame HDF5 dataset to multiple multipage grayscale TIFF files,
    each containing up to max_frames_per_tiff frames. The output filenames are based on
    the input file name with an appended integer to indicate ordering.
    
    Parameters:
        dirPath (str): Directory path where the HDF5 file is located.
        filePath (str): Name of the HDF5 file.
        output_dir (str): Directory where the output TIFF files will be saved.
        max_frames_per_tiff (int): Maximum number of frames per TIFF file.
        chunk_size (int): Number of frames to process at a time.
    """
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    input_path = os.path.join(dirPath, filePath)
    base_name, _ = os.path.splitext(filePath)
    
    with h5py.File(input_path, 'r') as f_in:
        dset = f_in['data']
        total_frames, height, width = dset.shape
        print("Dataset shape:", dset.shape)
        
        # Determine the number of output files needed
        num_files = (total_frames + max_frames_per_tiff - 1) // max_frames_per_tiff
        
        for file_index in range(num_files):
            start_frame = file_index * max_frames_per_tiff
            end_frame = min((file_index + 1) * max_frames_per_tiff, total_frames)
            
            # Create output TIFF filename, e.g., "base_name_1.tiff", "base_name_2.tiff", etc.
            output_tiff = os.path.join(output_dir, f"{base_name}_{file_index+1}.tiff")
            writer = imageio.get_writer(output_tiff, format='TIFF', mode='I')
            print(f"Writing frames {start_frame} to {end_frame-1} to {output_tiff}")
            
            # Process the current block in chunks
            for chunk_start in range(start_frame, end_frame, chunk_size):
                chunk_end = min(chunk_start + chunk_size, end_frame)
                for i in range(chunk_start, chunk_end):
                    frame = dset[i, :, :]
                    # Normalize to uint8 if needed
                    if frame.dtype != np.uint8:
                        if frame.max() == frame.min():
                            # If the frame is constant, set it to zeros
                            frame = np.zeros_like(frame, dtype=np.uint8)
                        else:
                            frame = ((frame - frame.min()) / (frame.max() - frame.min()) * 255).astype(np.uint8)
                    writer.append_data(frame)
                    print(f"Processed frame {i+1} of {total_frames}")
            writer.close()
            print(f"Finished writing {output_tiff}")
    
    print("All TIFF files have been saved.")
    
def tiffs_to_h5(tiff_dir, output_h5, dataset_name='data', chunk_size=100, swap_axes=False):
    """
    Combine multiple TIFF files from a directory into a single HDF5 dataset.
    This function reads each TIFF file in chunks to conserve memory.
    
    Before creating the HDF5 file, it scans all TIFF files to determine a common frame
    shape (after applying optional axis swapping) and the total number of frames.
    
    Parameters:
        tiff_dir (str): Directory containing the TIFF files.
        output_h5 (str): Path to the output HDF5 file.
        dataset_name (str): Name of the dataset to create in the HDF5 file.
        chunk_size (int): Number of frames to process at a time.
        swap_axes (bool): If True, transpose each frame (swap first two axes).
                          Use this if the loaded frame dimensions do not match expectations.
    """
    # List all TIFF files in the directory
    tiff_files = sorted(glob.glob(os.path.join(tiff_dir, "*.tif")))
    if not tiff_files:
        raise ValueError(f"No TIFF files found in {tiff_dir}")
    
    total_frames = 0
    common_shape = None  # expected (height, width)
    file_info = []       # list of tuples: (filename, n_frames)
    
    # First pass: determine total frames and common frame shape.
    for tiff_file in tiff_files:
        reader = imageio.get_reader(tiff_file, format='TIFF')
        frame0 = reader.get_data(0)
        if swap_axes:
            frame0 = frame0.T
        # Determine shape and number of frames for this file.
        if frame0.ndim == 2:
            # Multipage TIFF: each page is one 2D frame.
            shape = frame0.shape  # (height, width)
            n_frames_file = reader.get_length()
        elif frame0.ndim == 3:
            # 3D stack stored in one page; assume shape is (height, width, n_slices)
            shape = frame0.shape[:2]
            if reader.get_length() == 1:
                n_frames_file = frame0.shape[2]
            else:
                # In the uncommon case there are multiple pages of 3D stacks,
                # treat each page as one volume and use the first slice's shape.
                n_frames_file = reader.get_length()
                shape = reader.get_data(0).shape[:2]
        else:
            reader.close()
            raise ValueError(f"Unexpected frame dimensions in file: {tiff_file}")
        
        # If common_shape hasn't been set yet, use this file's shape.
        if common_shape is None:
            common_shape = shape
        else:
            # If a file's shape does not match, raise an error (or choose how to resolve it).
            if common_shape != shape:
                reader.close()
                raise ValueError(f"Inconsistent frame shapes: expected {common_shape} but got {shape} in {tiff_file}")
        
        total_frames += n_frames_file
        file_info.append((tiff_file, n_frames_file))
        reader.close()
    
    print(f"Total frames: {total_frames}, Common frame size: {common_shape[0]}x{common_shape[1]}")
    
    # Create the HDF5 file and dataset.
    with h5py.File(output_h5, 'w') as f_out:
        dset = f_out.create_dataset(dataset_name, shape=(total_frames, common_shape[0], common_shape[1]),
                                    dtype=np.uint8, chunks=True)
        current_index = 0
        
        # Second pass: process each file and write frames into the HDF5 dataset.
        for tiff_file, n_frames_file in file_info:
            reader = imageio.get_reader(tiff_file, format='TIFF')
            frame0 = reader.get_data(0)
            if swap_axes:
                frame0 = frame0.T
            # Process files with 2D pages (multipage TIFF)
            if frame0.ndim == 2:
                for start in range(0, n_frames_file, chunk_size):
                    end = min(start + chunk_size, n_frames_file)
                    frames = []
                    for i in range(start, end):
                        frame = reader.get_data(i)
                        if swap_axes:
                            frame = frame.T
                        # Normalize to uint8 if necessary.
                        if frame.dtype != np.uint8:
                            frame = ((frame - frame.min()) / (frame.max() - frame.min() + 1e-8) * 255).astype(np.uint8)
                        frames.append(frame)
                    frames = np.array(frames)
                    num_chunk = frames.shape[0]
                    dset[current_index:current_index+num_chunk, :, :] = frames
                    current_index += num_chunk
                    print(f"Processed frames {current_index}/{total_frames} from {tiff_file}")
            # Process files with a single 3D stack.
            elif frame0.ndim == 3:
                if reader.get_length() == 1:
                    # Assume the stack shape is (height, width, n_slices)
                    n_slices = frame0.shape[2]
                    for start in range(0, n_slices, chunk_size):
                        end = min(start + chunk_size, n_slices)
                        frames = frame0[:, :, start:end]
                        # Convert from (height, width, n_slices) to (n_slices, height, width)
                        frames = np.transpose(frames, (2, 0, 1))
                        if swap_axes:
                            frames = np.array([f.T for f in frames])
                        if frames.dtype != np.uint8:
                            frames = np.array([((f - f.min()) / (f.max() - f.min() + 1e-8)*255).astype(np.uint8) for f in frames])
                        num_chunk = frames.shape[0]
                        dset[current_index:current_index+num_chunk, :, :] = frames
                        current_index += num_chunk
                        print(f"Processed frames {current_index}/{total_frames} from {tiff_file}")
                else:
                    # Handle an unlikely case of multiple pages with 3D stacks.
                    for start in range(0, n_frames_file, chunk_size):
                        end = min(start + chunk_size, n_frames_file)
                        frames = []
                        for i in range(start, end):
                            frame = reader.get_data(i)
                            if swap_axes:
                                frame = frame.T
                            # If the frame is still 3D, take the first slice.
                            if frame.ndim == 3:
                                frame = frame[:, :, 0]
                            if frame.dtype != np.uint8:
                                frame = ((frame - frame.min()) / (frame.max() - frame.min() + 1e-8) * 255).astype(np.uint8)
                            frames.append(frame)
                        frames = np.array(frames)
                        num_chunk = frames.shape[0]
                        dset[current_index:current_index+num_chunk, :, :] = frames
                        current_index += num_chunk
                        print(f"Processed frames {current_index}/{total_frames} from {tiff_file}")
            else:
                reader.close()
                raise ValueError(f"Unexpected frame dimensions in file: {tiff_file}")
            reader.close()
    print(f"All {total_frames} frames have been saved to {output_h5}")

In [7]:
dirPath = '/scratch/jma819/scope_data/SPRT/m1_GRAB_analysis/'
filePath = '2024-11-12-15-23-47_channel1.h5'
output_avi = dirPath+'2024-11-12-15-23-47_channel1.avi'     # output AVI filename
output_dir = '/scratch/jma819/scope_data/SPRT/m1_GRAB_analysis/2024-11-12-15-23-47_channel1_tiff_output/'
output_tiff = dirPath+'2024-11-12-15-23-47_channel1.tiff'
fps = 20

In [8]:
h5_to_tiff_split(dirPath, filePath, output_dir, max_frames_per_tiff=1000, chunk_size=100)

Dataset shape: (3204, 1280, 800)
Writing frames 0 to 999 to /scratch/jma819/scope_data/SPRT/m1_GRAB_analysis/2024-11-12-15-23-47_channel1_tiff_output/2024-11-12-15-23-47_channel1_1.tiff
Processed frame 1 of 3204
Processed frame 2 of 3204
Processed frame 3 of 3204
Processed frame 4 of 3204
Processed frame 5 of 3204
Processed frame 6 of 3204
Processed frame 7 of 3204
Processed frame 8 of 3204
Processed frame 9 of 3204
Processed frame 10 of 3204
Processed frame 11 of 3204
Processed frame 12 of 3204
Processed frame 13 of 3204
Processed frame 14 of 3204
Processed frame 15 of 3204
Processed frame 16 of 3204
Processed frame 17 of 3204
Processed frame 18 of 3204
Processed frame 19 of 3204
Processed frame 20 of 3204
Processed frame 21 of 3204
Processed frame 22 of 3204
Processed frame 23 of 3204
Processed frame 24 of 3204
Processed frame 25 of 3204
Processed frame 26 of 3204
Processed frame 27 of 3204
Processed frame 28 of 3204
Processed frame 29 of 3204
Processed frame 30 of 3204
Processed fra

In [9]:
# Open the HDF5 file and read dataset shape
with h5py.File(dirPath + filePath, 'r') as f_in:
    dset = f_in['data']
    total_frames, height, width = dset.shape
    print("Dataset shape:", dset.shape)

    # Create an imageio writer; codec "rawvideo" should give you an uncompressed output
    writer = imageio.get_writer(output_avi, fps=fps, codec='rawvideo')
    
    for i in range(total_frames):
        frame = dset[i, :, :]
        # If the frame is not uint8, scale it appropriately.
        if frame.dtype != np.uint8:
            frame = ((frame - frame.min()) / (frame.max() - frame.min()) * 255).astype(np.uint8)
        writer.append_data(frame)
    writer.close()

print(f"Saved {total_frames} frames to {output_avi} as an uncompressed AVI.")


Dataset shape: (3204, 1280, 800)
Saved 3204 frames to /scratch/jma819/scope_data/SPRT/m1_GRAB_analysis/2024-11-12-15-23-47_channel1.avi as an uncompressed AVI.


In [2]:
# after motion correction convert tiffs back to h5

# get tiff files
tiff_dir = "/scratch/jma819/scope_data/SPRT/m1_GRAB_analysis/2024-11-12-15-23-47_channel1_tiff_output/motion_corrected"
tiff_files = sorted(glob.glob(os.path.join(tiff_dir, "*.tif")))

In [3]:
tiff_files

['/scratch/jma819/scope_data/SPRT/m1_GRAB_analysis/2024-11-12-15-23-47_channel1_tiff_output/motion_corrected/2024-11-12-15-23-47_channel1_tiff_output2024-11-12-15-23-47_channel1_01___motion_corrected.tif',
 '/scratch/jma819/scope_data/SPRT/m1_GRAB_analysis/2024-11-12-15-23-47_channel1_tiff_output/motion_corrected/2024-11-12-15-23-47_channel1_tiff_output2024-11-12-15-23-47_channel1_02___motion_corrected.tif',
 '/scratch/jma819/scope_data/SPRT/m1_GRAB_analysis/2024-11-12-15-23-47_channel1_tiff_output/motion_corrected/2024-11-12-15-23-47_channel1_tiff_output2024-11-12-15-23-47_channel1_03___motion_corrected.tif',
 '/scratch/jma819/scope_data/SPRT/m1_GRAB_analysis/2024-11-12-15-23-47_channel1_tiff_output/motion_corrected/2024-11-12-15-23-47_channel1_tiff_output2024-11-12-15-23-47_channel1_04___motion_corrected.tif']

In [14]:
output_h5 = '/scratch/jma819/scope_data/SPRT/m1_GRAB_analysis/2024-11-12-15-23-47_channel1_tiff_output/motion_corrected_combined_dataset.h5'
tiffs_to_h5(tiff_dir, output_h5, dataset_name='data', chunk_size=100, swap_axes=False)

ValueError: Inconsistent frame shapes: expected (1000, 1280) but got (204, 1280) in /scratch/jma819/scope_data/SPRT/m1_GRAB_analysis/2024-11-12-15-23-47_channel1_tiff_output/motion_corrected/2024-11-12-15-23-47_channel1_tiff_output2024-11-12-15-23-47_channel1_04___motion_corrected.tif

In [17]:
tiff_dir = '/Users/johnmarshall/Documents/Analysis/nVueData/SPRT/SPRT_m1_d6/2024-11-12-15-23-47_channel1_tiff_output/'
tiff_files = sorted(glob.glob(os.path.join(tiff_dir, "*.tif")))

file_info = []       # list of tuples: (filename, n_frames)
totalFrames = 0; 
for tiff_file in tiff_files:
    reader = imageio.get_reader(tiff_file, format='TIFF')
    n_pages = reader.get_length()
    totalFrames = totalFrames + n_pages
    frame0 = reader.get_data(0)
    print(f"File: {os.path.basename(tiff_file)}")
    print(f"  Number of pages/frames: {n_pages}")
    print(f"  First frame shape: {frame0.shape}\n")
    reader.close()
    common_shape = (frame0.shape)
    file_info.append((tiff_file, n_pages))
totalFrames   

File: 2024-11-12-15-23-47_channel1_01.tif
  Number of pages/frames: 1000
  First frame shape: (1280, 800)

File: 2024-11-12-15-23-47_channel1_02.tif
  Number of pages/frames: 1000
  First frame shape: (1280, 800)

File: 2024-11-12-15-23-47_channel1_03.tif
  Number of pages/frames: 1000
  First frame shape: (1280, 800)

File: 2024-11-12-15-23-47_channel1_04.tif
  Number of pages/frames: 204
  First frame shape: (1280, 800)



3204

In [43]:
chunk_size = 100
output_h5 = '/Users/johnmarshall/Documents/Analysis/nVueData/SPRT/SPRT_m1_d6/2024-11-12-15-23-47_channel1_tiff_output/h5_outTest.h5'

current_index = 0
with h5py.File(output_h5, 'w') as f_out:
    dataset_name = 'data' 
    dset = f_out.create_dataset(dataset_name, shape=(totalFrames, common_shape[0], common_shape[1]),
                                    dtype=np.uint8, chunks=True)
    for tiff_file, n_frames_file in file_info:
        reader = imageio.get_reader(tiff_file, format='TIFF')
        frame0 = reader.get_data(0)
        n_slices = n_frames_file
        for start in range(0, n_slices, chunk_size):
            end = min(start + chunk_size, n_slices)
            frames = np.array([reader.get_data(i) for i in range(start, end)])
            num_chunk = frames.shape[0]
            dset[current_index:current_index+num_chunk, :, :] = frames
            current_index += num_chunk

In [41]:
np.shape(frames)

(4, 1280, 800)

In [42]:
frames.dtype

dtype('uint8')

In [38]:
end

204